# Лабораторная 2

In [1]:
from abc import ABCMeta, abstractmethod
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("data\creditcard.csv")

In [3]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [4]:
class Classificator():
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def train(X, Y):
        '''Обучить модель'''
    
    @abstractmethod
    def predict(self, X):
        '''Предсказать данные'''

In [16]:
class DecisionTree(Classificator):
    model = None
    node = {}
    
    def train(self, X, Y, tabs="", depth = 0, prediction=None, max_depth=None, leafValue=None):
        entropy = self.calculate_entropy(Y)
        #print(entropy)
        #print(Y.nunique())
        #print(X.columns)
        
        if entropy == 0 or depth == max_depth or len(X) == leafValue:
            p = len(Y.loc[Y == 1]) / len(Y)
            #self.node[prediction.attribute] = [prediction.predicate, p]
            self.node['label'] = p
            return self.node
        igs = []
        for index, row in X[:1].iterrows():
            inner_igs = []
            for column in X:
                predicate = row[column]
                left = X[column].loc[X[column] <= predicate]
                right = X[column].loc[X[column] > predicate]
                Y_left = Y.loc[left.index]
                Y_right = Y.loc[right.index]
                groups = [[len(left), self.calculate_entropy(Y_left)], [len(right), self.calculate_entropy(Y_right)]]
                ig = self.information_gain(df.Class, entropy, groups)
                inner_igs.append(Predicate(column, ig, predicate))
            igs.append(self.get_max_ig(inner_igs))

        predicate = self.get_max_ig(igs)
        
        X_left = X.loc[X[predicate.attribute] <= predicate.predicate].drop(predicate.attribute, axis=1)
        X_right = X.loc[X[predicate.attribute] > predicate.predicate].drop(predicate.attribute, axis=1)
        Y_left = Y.loc[X_left.index]
        Y_right = Y.loc[X_right.index]
        depth += 1
        #print(tabs + "left")
        #left_node = DecisionTree().train(X_left, Y_left, prediction=prediction depth=depth, max_depth=max_depth)
        #print(tabs + "right")
        #right_node = DecisionTree().train(X_right, Y_right, prediction=prediction depth=depth, max_depth=max_depth)
        #left = DecisionTree().train(X_left, Y_left, prediction=predicate, depth=depth, max_depth=max_depth)
        #rigth = DecisionTree().train(X_right, Y_right, prediction=predicate, depth=depth, max_depth=max_depth)
        self.node['attribute'] = predicate.attribute
        self.node['value'] = predicate.predicate
        self.node['nodes'] = {}
        self.node['nodes']['<='] = DecisionTree().train(X_left, Y_left, prediction=predicate, depth=depth, max_depth=max_depth)
        self.node['nodes']['>'] = DecisionTree().train(X_right, Y_right, prediction=predicate, depth=depth, max_depth=max_depth)
        #self.node[predicate.attribute] = predicate.predicate
        return self.node
    
    def predict(self, X):
        pass
    
    def calculate_entropy(self, Y):
        count = len(Y)
        entropy = 0
        for count_class in Y.value_counts():
            p = count_class / count
            entropy -= p * np.log2(p)
        return entropy
    
    
    def information_gain(self, Y, entropy0, groups):
        N = len(Y)
        sum_entropies = 0
        for group in groups:
            group
            Ni = group[0]
            Si = group[1]
            sum_entropies += Ni / N * Si
        return entropy0 - sum_entropies
    
    def get_max_ig(self, igs):
        information_gain = igs[0]
        for ig in igs:
            if ig.ig > information_gain.ig:
                information_gain = ig
        return information_gain

In [7]:
class RandomForest(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [8]:
class GradientBoosting(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [20]:
X = df[df.columns[:-1]]
y = df.Class

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTree()
tree.train(X_train, Y_train)
read_tree(tree.node)
#print(pred.attribute, pred.ig)

V12


TypeError: can only concatenate str (not "numpy.float64") to str

In [23]:
def read_tree(tree, tabs=''):
    print(tabs+tree['attribute'])
    print(tabs+str(tree['value']))
    print(tabs+str(tree['label']))
    print(tree['nodes']['<='])
    print(tree['nodes']['>'])
    #read_tree(tree['nodes']['<='], tabs+'\t')
    #read_tree(tree['nodes']['>'], tabs+'\t')
    return

In [5]:
class Predicate:
    attribute = ""
    ig = 0
    predicate = 0
    
    def __init__(self, attribute, ig, predicate):
        self.attribute = attribute
        self.ig = ig
        self.predicate = predicate

In [23]:
len(df.Class.loc[df.Class == 1])

492

In [24]:
read_tree(tree.node)

V12
-2.0438683867492897
0.0
{'attribute': 'V12', 'value': -2.0438683867492897, 'nodes': {'<=': {...}, '>': {...}}, 'label': 0.0}
{'attribute': 'V12', 'value': -2.0438683867492897, 'nodes': {'<=': {...}, '>': {...}}, 'label': 0.0}
